In [1]:
import os
import pyltp
import time
import pandas as pd
import numpy as np

In [2]:
%%time
articles = pd.read_csv("articles.csv",encoding="gbk")
LTP_DIR = "./model/ltp_data_v3.4.0"
# print(articles)

#分词模型
segmentor = pyltp.Segmentor()
segmentor.load(os.path.join(LTP_DIR, "cws.model"))

#词性模型
postagger = pyltp.Postagger()
postagger.load(os.path.join(LTP_DIR, 'pos.model'))

#命名实体模型
recognizer = pyltp.NamedEntityRecognizer()
recognizer.load(os.path.join(LTP_DIR, 'ner.model'))

#依存句法分析
parser = pyltp.Parser() 
parser.load(os.path.join(LTP_DIR, 'parser.model'))  


Wall time: 24.8 s


In [4]:
%%time
# sentences(
#  doc_id text,
#  sentence_index int,
#  sentence_text text,
#  tokens text[],
#  lemmas text[],
#  pos_tags text[],
#  ner_tags text[],
#  doc_offsets int[],
#  dep_types text[],
#  dep_tokens int[]
# ).

sentences = []
article_number = articles.shape[0]


for article in range(article_number):
    #分句
    sents = pyltp.SentenceSplitter.split(articles.iloc[article]['text'])
    
#     token_offset = 0
    for index,sent in enumerate(list(sents)):
        #doc_id:文章id
        sentence = str(articles.iloc[article]['id'])+"&&&&"
        #sentence_index:句子id
        sentence = sentence+str(index+1)+"&&&&"
        #sentence_text
        sentence = sentence+sent+"&&&&"
        #tokens：分词
        tokens = list(segmentor.segment(sent))
        sentence = sentence+'@@'.join(tokens)+"&&&&"
        
        #lemmas：词元
        
        #pos_tags:词性
        pos_tags = list(postagger.postag(tokens))
        sentence = sentence+'@@'.join(pos_tags)+"&&&&"
        #ner_tags:命名实体
        ner_tags = list(recognizer.recognize(tokens, pos_tags))
        sentence = sentence+'@@'.join(ner_tags)+"&&&&"
        
#         #doc_offsets:单词偏移量
#         token_length = list(map(lambda x:len(x),tokens))
#         token_length.insert(0, token_offset)
#         token_length = np.array(token_length)
#         doc_offset = token_length.cumsum(0)
#         sentence.append(list(doc_offset[:-1]))
#         token_offset = doc_offset[-1]

        #dep_types，dep_tokens：依存句法
        arcs = parser.parse(tokens, pos_tags)
        arcs = np.array(list(map(lambda x: [x.head,x.relation],arcs)))
        sentence = sentence+'@@'.join(list(arcs[:,1]))+"&&&&"
        sentence = sentence+'@@'.join(list(arcs[:,0]))  
        #一个和deepdive相同的完整的sentence的结构
        
        sentences.append(sentence)

sentences = {'sentences_list':sentences}        
df = pd.DataFrame(sentences,columns = ['sentences_list'])        
df.to_csv("sentences_nlp.csv",index=0,header=0,encoding="UTF-8")
        
segmentor.release()
postagger.release()
recognizer.release()
parser.release()

Wall time: 14.9 s
